In [78]:
import sys
print(sys.version)
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

3.10.11 | packaged by Anaconda, Inc. | (main, Apr 20 2023, 18:56:50) [MSC v.1916 64 bit (AMD64)]


In [79]:
!pip install openpyxl

In [80]:
df = pd.read_excel("Observability-TechTest-2022-feb.xlsx")

df.head(5)

,day,proposition,viewerID,asset,deviceOS,country,state,city,asn,isp,...,session_out_of_home,session_channel_id,session_content_type,session_episode,session_goal_id,session_viewer_id,session_pt_os,session_pt_os_ver,session_device_marketing_name,session_player_name
0,2020-08-19,SkyDE,192283420a01f97a2c1f4c936aa4d1b2a9c5439b,Live-129-Sky Sport 1 HD,iOS,austria,niederoesterreich,maria enzersdorf,8339,"ASN: KABSI-AS, AT",...,1.0,NaN,unknown,NaN,NaN,NaN,iOS,iOS 13.6,Apple iPhone 8,skygo-de-ios
1,2020-08-19,SkyDE,1396c74a853f8651bf418c55296b3c715deeb0a1,Live-129-Sky Sport 1 HD,OS X,germany,hamburg,hamburg,3209,Vodafone,...,1.0,NaN,NaN,NaN,NaN,NaN,macOS,OS X 10.15.6,"Apple MacBookAir7,2",skygo-de-desktop
2,2020-08-19,SkyDE,015f3b9bb5dca429f106796d61b2aa0dd86e8dcd,Live-129-Sky Sport 1 HD,Tizen,germany,hamburg,hamburg,31334,Kable Deutschland,...,NaN,NaN,Movies,NaN,NaN,NaN,Tizen,Tizen Platform: 5.0 | API: 3.0 | Web API: 3.0,Samsung Tizen TV,skyq-de-samsungtv
3,2020-08-19,SkyDE,d10afbbb7a2ba45c0c9de4765d4aa1f10fbfd4e7,Live-82286d1e-1858-11e9-bf44-db93f4694cd7-Sky ...,tvOS,germany,baden-wuerttemberg,karlsruhe,8881,Versatel,...,NaN,NaN,NaN,NaN,NaN,NaN,tvOS,tvOS 13.4.8,Apple Unknown,skyticket-de-appletv
4,2020-08-19,SkyDE,94714557f7f8da09b815b96b1c0574d8963485a8,Live-129-Sky Sport 1 HD,iOS,germany,bremen,bremen,3320,Deutsche Telekom,...,1.0,NaN,unknown,NaN,NaN,NaN,iOS,iOS 12.4.8,Apple iPad Air,skygo-de-ios


In [81]:
# Dataset contains 41 columns and over 38 thousand records

df.shape

(38733, 41)

In [82]:
df.columns.to_list()

['day',
 'proposition',
 'viewerID',
 'asset',
 'deviceOS',
 'country',
 'state',
 'city',
 'asn',
 'isp',
 'startTime',
 'startupTimeMS',
 'playingTimeMS',
 'interrupts',
 'startupError',
 'cdn',
 'browser',
 'vpf',
 'vpfErrorList',
 'networkType',
 'endTime',
 'session_device_manufacturer',
 'session_device_brand',
 'session_device_conn',
 'session_appversion',
 'session_browser_version',
 'session_hardware_type',
 'session_source',
 'session_device_model',
 'session_is_live',
 'session_channel_name',
 'session_out_of_home',
 'session_channel_id',
 'session_content_type',
 'session_episode',
 'session_goal_id',
 'session_viewer_id',
 'session_pt_os',
 'session_pt_os_ver',
 'session_device_marketing_name',
 'session_player_name']

In [83]:
# Start time from 20:20:00 to 20:40:00

df["startTime"].min()


'2020-08-19 20:20:00 UTC'

In [84]:
df["startTime"].max()

'2020-08-19 20:40:00 UTC'

In [85]:
df["startTime"].dtype

dtype('O')

In [86]:
# Has any issues occured during start up session ? 
# No, there was no issues reported during start up

df["startupError"].unique()

array([0], dtype=int64)

In [87]:
# Has Video Play Failure (VPF) always occured ? 
# Yes, every single user experienced issues

df["vpf"].unique()

array([1], dtype=int64)

In [88]:
# What kind of network viewers used during transmission ? 
# All records indicates external network only

df["networkType"].unique()

array(['external'], dtype=object)

In [89]:
# Some of the columns wont have impact on analysis, hence Im droping them to reduce disk space

columns_to_drop =["proposition", "viewerID", "asn", "startupError", "vpf", "networkType", 
                  "session_device_brand", "session_appversion", "session_browser_version",
                  "session_source", "session_device_model", "session_is_live", "session_out_of_home", 
                  "session_channel_id", "session_content_type","session_episode", 
                  "session_goal_id", "session_viewer_id", "session_device_marketing_name"] 

df.drop(columns=columns_to_drop, inplace = True)

In [90]:
# Dataset has been compacted to 22 columns

df.shape

(38733, 22)

In [91]:
df.columns.to_list()

['day',
 'asset',
 'deviceOS',
 'country',
 'state',
 'city',
 'isp',
 'startTime',
 'startupTimeMS',
 'playingTimeMS',
 'interrupts',
 'cdn',
 'browser',
 'vpfErrorList',
 'endTime',
 'session_device_manufacturer',
 'session_device_conn',
 'session_hardware_type',
 'session_channel_name',
 'session_pt_os',
 'session_pt_os_ver',
 'session_player_name']

In [92]:
# converting complex startTime column into time only

df["startTime"] = pd.to_datetime(df["startTime"])


df["convertedStartTime"] = df["startTime"].dt.strftime("%H:%M:%S")

In [93]:
df["convertedStartTime"]

0        20:20:00
1        20:20:00
2        20:20:00
3        20:20:00
4        20:20:00
           ...   
38728    20:40:00
38729    20:40:00
38730    20:40:00
38731    20:40:00
38732    20:40:00
Name: convertedStartTime, Length: 38733, dtype: object

In [94]:
# converting complex endTime column into time type

df["endTime"]= pd.to_datetime(df["endTime"])

df["convertedEndTime"] = df["endTime"].dt.strftime("%H:%M:%S")


In [95]:
df["convertedEndTime"]

0        21:24:03
1        20:29:43
2        20:30:31
3        20:20:11
4        20:38:01
           ...   
38728    21:12:01
38729    20:40:50
38730    20:40:50
38731    20:40:50
38732    20:40:50
Name: convertedEndTime, Length: 38733, dtype: object

In [96]:
df.head(3)

,day,asset,deviceOS,country,state,city,isp,startTime,startupTimeMS,playingTimeMS,...,endTime,session_device_manufacturer,session_device_conn,session_hardware_type,session_channel_name,session_pt_os,session_pt_os_ver,session_player_name,convertedStartTime,convertedEndTime
0,2020-08-19,Live-129-Sky Sport 1 HD,iOS,austria,niederoesterreich,maria enzersdorf,"ASN: KABSI-AS, AT",2020-08-19 20:20:00+00:00,2258,3843094,...,2020-08-19 21:24:03.094000+00:00,Apple,WiFi,Mobile Phone,Sky Sport 1 HD,iOS,iOS 13.6,skygo-de-ios,20:20:00,21:24:03
1,2020-08-19,Live-129-Sky Sport 1 HD,OS X,germany,hamburg,hamburg,Vodafone,2020-08-19 20:20:00+00:00,6406,583780,...,2020-08-19 20:29:43.780000+00:00,Apple,Unknown,Desktop,Sky Sport 1 HD,macOS,OS X 10.15.6,skygo-de-desktop,20:20:00,20:29:43
2,2020-08-19,Live-129-Sky Sport 1 HD,Tizen,germany,hamburg,hamburg,Kable Deutschland,2020-08-19 20:20:00+00:00,1767,618627,...,2020-08-19 20:30:31.329000+00:00,Samsung,Unknown,TV,Sky Sport 1 HD,Tizen,Tizen Platform: 5.0 | API: 3.0 | Web API: 3.0,skyq-de-samsungtv,20:20:00,20:30:31


In [97]:
# Convert startTime and endTime columns to datetime objects as I was unable to save new excel file

df["startTime"] = pd.to_datetime(df["startTime"])
df["endTime"] = pd.to_datetime(df["endTime"])


In [98]:
# Remove timezones from startTime and endTime columns 

df['startTime'] = df['startTime'].dt.tz_localize(None)
df['endTime'] = df['endTime'].dt.tz_localize(None)

In [99]:
cleaned_observability = pd.DataFrame(df)

cleaned_observability.to_excel("C:/Users/katar_223g2eq/OneDrive/Katarina_Data_Science/Data Science/Github/Observability/cleaned_observability.xlsx", index = False)